In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from cascade.cascade_driver import CascadeDriver
cas_driver = CascadeDriver(zenith_angle = 30)

In [ ]:
from tqdm import tqdm
# mceq_decaying_pdgs = [111, -211, 211]

# mceq_decaying_pdgs = [-11, 11, -12, 12, -14, 14, 16, 22,
#                         111, 130, -211, 211, 310,
#                         -321, 321, -2112, 2112, 
#                         -2212, 2212, -3122, 3122]

# mceq_decaying_pdgs = [111, 130, 310, -13, 13, -211, 211]
# mceq_decaying_pdgs = [111, 130, 310]

mceq_decaying_pdgs = [-11, 11, -12, 12, -13, 13, -14, 14, 
                      -16, 16, 22, 
                        111, 130, -211, 211, 310, -321, 321, 
                        -411, 411, -421, 421, -431, 431, 
                        -2112, 2112, -2212, 2212, -3122, 3122
                        ]


cas_driver.simulation_parameters(pdg = 2212, energy = 1e2, xdepth = 0,
                                 threshold_energy = 1e-6, stop_height = 5,
                                 accumulate_runs = True, reset_ids = True,
                                 mceq_decaying_pdgs = mceq_decaying_pdgs)

niter = 10000
for i in tqdm(range(niter), total = niter):
    cas_driver.run()

In [ ]:
from cascade_analysis import CascadeAnalysis

cascade_analysis = CascadeAnalysis(cas_driver)
cascade_analysis.print_stats()
cascade_analysis.search_for_parents()

In [ ]:
# cascade_analysis.check_particle_existence()

In [ ]:
cascade_analysis.plot_ptypes_dist(from_ = 1)

In [ ]:
cascade_analysis.plot_ptypes_energy_dist(from_ = 0)

In [ ]:
cascade_analysis.plot_energy()

In [ ]:
cascade_analysis.plot_energy_list(pids = [-13, 13, -14, 14, -11, 11, 2212, 2112, -211, 211])

In [ ]:
%autoreload 2
from mceq_comparison import MCEQDistributions
import matplotlib.pylab as plt
mceq_noloss = MCEQDistributions(
                 energy = 1e2,
                 pdg_id = 2212,
                 theta_deg = 30,
                 slant_depth = 800,
                #  slant_depth = 635.9540964571235,
                 pname_tuples = [
                     ("mu", "mu+", "mu-"),
                     ("numu", "numu", "antinumu"),
                     ("nue", "nue", "antinue"),
                     ("pi", "pi+", "pi-")],
                 interaction_model = "DPMJET-III-19.1",
                 generic_losses_all_charged = False, 
                 enable_energy_loss = False, 
                 muon_helicity_dependence = False,
                 disable_decays = [],
                 density_model = ("CORSIKA", ("BK_USStd", None)))
                #  disable_decays = [-13, 13, -211, 211]
mceq_dist = mceq_noloss

In [ ]:
%autoreload 2
mceq_loss = MCEQDistributions(
                 energy = 1e2,
                 pdg_id = 2212,
                 theta_deg = 30,
                #  slant_depth = 635.9540964571235,
                 slant_depth = 638,
                 pname_tuples = [
                     ("mu", "mu+", "mu-"),
                     ("numu", "numu", "antinumu"),
                     ("nue", "nue", "antinue"),
                     ("pi", "pi+", "pi-")],
                 interaction_model = "DPMJET-III-19.1",
                #  generic_losses_all_charged = False, 
                 enable_energy_loss = True, 
                 muon_helicity_dependence = True,
                 disable_decays = [],
                 density_model = ("CORSIKA", ("USStd", None)))



In [ ]:
mceq_loss_bk = MCEQDistributions(
                 energy = 1e2,
                 pdg_id = 2212,
                 theta_deg = 30,
                #  slant_depth = 635.9540964571235,
                 slant_depth = 638,
                 pname_tuples = [
                     ("mu", "mu+", "mu-"),
                     ("numu", "numu", "antinumu"),
                     ("nue", "nue", "antinue"),
                     ("pi", "pi+", "pi-")],
                 interaction_model = "DPMJET-III-19.1",
                #  generic_losses_all_charged = False, 
                 enable_energy_loss = True, 
                 muon_helicity_dependence = True,
                 disable_decays = [],
                 density_model = ("CORSIKA", ("BK_USStd", None)))

In [ ]:

cascade_analysis.search_for_parents()

In [ ]:
plt.rcParams["figure.dpi"] = 150

bins, hist, label = cascade_analysis.kin_energy_histogram(pdgs = (-13, 13), bins = mceq_dist.e_bins)
plt.stairs(hist, bins, label = label, linestyle='-')

bins, hist, label = cascade_analysis.kin_energy_histogram(pdgs = (-12, -12), bins = mceq_dist.e_bins)
plt.stairs(hist, bins, label = label, linestyle='-')

bins, hist, label = cascade_analysis.kin_energy_histogram(pdgs = (-14, 14), bins = mceq_dist.e_bins)
plt.stairs(hist, bins, label = label, linestyle='-')

bins, hist, label = cascade_analysis.kin_energy_histogram(pdgs = (-211, 211), bins = mceq_dist.e_bins)
plt.stairs(hist, bins, label = label, linestyle='-')


plt.stairs(mceq_dist.flux["mu"], mceq_dist.e_bins, 
           label = r"${\mu}^{+} + {\mu}^{-}$ mceq", linestyle='--')
plt.stairs(mceq_dist.flux["numu"], mceq_dist.e_bins, 
           label = r"$\bar{\nu}_{\mu} + {\nu}_{\mu}$ mceq", linestyle='--')
plt.stairs(mceq_dist.flux["nue"], mceq_dist.e_bins, 
           label = r"$\bar{\nu}_{e} + {\nu}_{e}$ mceq", linestyle='--')
plt.stairs(mceq_dist.flux["pi"], mceq_dist.e_bins, 
           label = r"$\bar{\pi}^{+} + {\pi}^{-}$ mceq", linestyle='--')

plt.xscale("log")
plt.xlim(1e-2, 2e2)
# plt.ylim(-0.01, 0.5)
plt.legend()
plt.grid()
# plt.savefig('temp.png', transparent=True)

In [ ]:
bins, hist, label = cascade_analysis.kin_energy_histogram(pdgs = (-13, 13), bins = mceq_dist.e_bins)
plt.stairs(hist, bins, label = label, linestyle='-')


plt.stairs(mceq_dist.flux["mu"], mceq_dist.e_bins, 
           label = r"${\mu}^{+} + {\mu}^{-}$ mceq no loss", linestyle='--')

# plt.stairs(mceq_loss.flux["mu"], mceq_loss.e_bins, 
#            label = r"${\mu}^{+} + {\mu}^{-}$ mceq loss", linestyle='--')

plt.xscale("log")
plt.xlim(1e-2, 2e2)
plt.ylim(-0.01, 0.5)
plt.legend()
plt.grid()

In [ ]:
bins, hist, label = cascade_analysis.kin_energy_histogram(pdgs = (-13, 13), bins = mceq_dist.e_bins)
plt.stairs(hist/mceq_dist.flux["mu"], bins, label = f"{label} cas/mceq", linestyle='-')

bins, hist, label = cascade_analysis.kin_energy_histogram(pdgs = (-12, 12), bins = mceq_dist.e_bins)
plt.stairs(hist/mceq_dist.flux["nue"], bins, label = f"{label} cas/mceq", linestyle='-')

bins, hist, label = cascade_analysis.kin_energy_histogram(pdgs = (-14, 14), bins = mceq_dist.e_bins)
plt.stairs(hist/mceq_dist.flux["numu"], bins, label = f"{label} cas/mceq", linestyle='-')


# plt.stairs(mceq_dist.flux["mu"]/1.1, mceq_dist.e_bins, 
#            label = r"${\mu}^{+} + {\mu}^{-}$ mceq no loss", linestyle='--')

# plt.stairs(mceq_loss.flux["mu"], mceq_loss.e_bins, 
#            label = r"${\mu}^{+} + {\mu}^{-}$ mceq loss", linestyle='--')

plt.xscale("log")
plt.xlim(1e-2, 3e2)
# plt.xlim(4e-1, 3e2)
plt.ylim(0.8, 1.2)
plt.legend()
plt.grid()

In [ ]:
plt.plot(mceq_loss.e_grid, mceq_loss.flux["mu"], 
         label = r"${\mu}^{+} + {\mu}^{-}$ USStd", linestyle='--')

# plt.plot(mceq_loss_bk.e_grid, mceq_loss_bk.flux["mu"], 
#          label = r"${\mu}^{+} + {\mu}^{-}$ BK_USStd", linestyle='-')

plt.xscale("log")
plt.xlim(4e-1, 2e2)
plt.ylim(-0.01, 0.31)
plt.legend()
plt.savefig('temp1.png', transparent=True)

In [ ]:
plt.plot(mceq_loss.e_grid, mceq_loss_bk.flux["mu"]/mceq_loss.flux["mu"], 
         label = r"${\mu}^{+} + {\mu}^{-}$ (BK_USStd/USStd)", linestyle='--')

# plt.plot(mceq_loss_bk.e_grid, mceq_loss_bk.flux["mu"], 
#          label = r"${\mu}^{+} + {\mu}^{-}$ bk", linestyle='-')

plt.xscale("log")
plt.xlim(4e-1, 2e2)
# plt.ylim(-0.01, 0.31)
plt.legend()
plt.savefig('temp.png', transparent=True)

In [ ]:
plt.plot(res_mceq.egrid, res_mceq.mu_spec[1], label = f"{res_mceq.mu_spec[2]}", linestyle='--')

plt.xscale("log")
plt.xlim(4e-1, 2e2)
plt.ylim(-0.01, 0.31)
plt.legend()
plt.savefig('temp.png', transparent=True)

In [ ]:
plt.plot(res_mceq.egrid, res_mceq.numu_spec[1], label = f"{res_mceq.numu_spec[2]} stairs", linestyle='--')

plt.xscale("log")
plt.xlim(4e-1, 2e2)
plt.ylim(-0.01, 2.5)
plt.legend()
plt.savefig('temp_line.png', transparent=True)

In [ ]:
plt.xscale("log")
plt.xlim(1e-1, 1e2)
plt.ylim(0.8, 1.2)
plt.step(res_mceq.mu_spec[0], cascade_analysis.mu/res_mceq.mu_spec[1], 
         label = r"${\mu}^{+} + {\mu}^{-}$ casc/mceq", linestyle='-', color = 'red')
plt.step(res_mceq.mu_spec[0], cascade_analysis.numu/res_mceq.numu_spec[1], 
         label = r"$\bar{\nu}_{\mu} + {\nu}_{\mu}$ casc/mceq", linestyle='-', color = 'green')
plt.step(res_mceq.mu_spec[0], cascade_analysis.nue/res_mceq.nue_spec[1], 
         label = r"$\bar{\nu}_{e} + {\nu}_{e}$ casc/mceq", linestyle='-', color = 'blue')
plt.step(res_mceq.mu_spec[0], res_mceq.nue_spec[1]/res_mceq.nue_spec[1], color = 'grey')
plt.legend()

In [ ]:
plt.xscale("log")
# plt.yscale("log")
plt.step(res_mceq.mu_spec[0], res_mceq.mu_spec[1], 
         label = r"${\mu}^{+} + {\mu}^{-}$ mceq", linestyle='--', color = 'red')
plt.step(res_mceq.mu_spec[0], cascade_analysis.mu, 
         label = r"${\mu}^{+} + {\mu}^{-}$ casc", linestyle='-', color = 'red')
plt.step(res_mceq.mu_spec[0], res_mceq.numu_spec[1], 
         label = r"$\bar{\nu}_{\mu} + {\nu}_{\mu}$ mceq", linestyle=':', color = 'green')
plt.step(res_mceq.mu_spec[0], cascade_analysis.numu, 
         label = r"$\bar{\nu}_{\mu} + {\nu}_{\mu}$ casc", linestyle='-', color = 'green')
plt.step(res_mceq.mu_spec[0], res_mceq.nue_spec[1], 
         label = r"$\bar{\nu}_{e} + {\nu}_{e}$ mceq", linestyle=':', color = 'blue')
plt.step(res_mceq.mu_spec[0], cascade_analysis.nue, 
         label = r"$\bar{\nu}_{e} + {\nu}_{e}$ casc", linestyle='-', color = 'blue')
# plt.step(res_mceq.numu_spec[0], res_mceq.numu_spec[1], label = res_mceq.numu_spec[2], linestyle='--')
# plt.step(res_mceq.nue_spec[0], res_mceq.nue_spec[1], label = res_mceq.nue_spec[2], linestyle='--')
plt.xlim(1e-1, 1e3)
# plt.ylim(0, 0.5)
plt.legend()

In [ ]:
plt.xscale("log")
plt.yscale("log")
plt.step(res_mceq.mu_spec[0], cascade_analysis.numu_from_mu, 
         label = r"$\bar{\nu}_{\mu} + {\nu}_{\mu}$ from muons", linestyle='--', color = 'green')
plt.step(res_mceq.mu_spec[0], cascade_analysis.numu_from_other, 
         label = r"$\bar{\nu}_{\mu} + {\nu}_{\mu}$ from other", linestyle=':', color = 'green')
plt.step(res_mceq.mu_spec[0], cascade_analysis.numu, 
         label = r"$\bar{\nu}_{\mu} + {\nu}_{\mu}$ total", linestyle='-', color = 'green')
plt.step(res_mceq.mu_spec[0], cascade_analysis.nue_from_mu, 
         label = r"$\bar{\nu}_{e} + {\nu}_{e}$ from muons", linestyle='--', color = 'blue')
plt.step(res_mceq.mu_spec[0], cascade_analysis.nue_from_other, 
         label = r"$\bar{\nu}_{e} + {\nu}_{e}$ from other", linestyle=':', color = 'blue')
plt.step(res_mceq.mu_spec[0], cascade_analysis.nue, 
         label = r"$\bar{\nu}_{e} + {\nu}_{e}$ total", linestyle='-', color = 'blue')
# plt.step(res_mceq.numu_spec[0], res_mceq.numu_spec[1], label = res_mceq.numu_spec[2], linestyle='--')
# plt.step(res_mceq.nue_spec[0], res_mceq.nue_spec[1], label = res_mceq.nue_spec[2], linestyle='--')
plt.xlim(1e-1, 1e3)
plt.legend()

In [ ]:
from fluka_comparison.fluka_muon_data import fluka_en_dist

In [ ]:
for name, value in fluka_en_dist().items():
    # plt.steps(value[0], value[1])
    plt.stairs(value[1], value[0], label = f"depth = {name}")
    en_bins = value[0]

gr, cnt = np.histogram(cascade_analysis.raw_muon_data[0], bins=en_bins)
gr = gr/cascade_analysis.raw_muon_data[1] 
print(cnt, gr)
plt.stairs(gr, cnt, label = "depth = 635.955, cas")
plt.legend()
plt.ylim(0, 2) 

# gr1, cnt = np.histogram(self.neutrinos_from_muons[muon_neut].energy, bins = nbins, range = xrange)
#         gr1 = gr1/runs_number
#         plt.step(mceq_egrid, gr1, lab

In [ ]:
cascade_analysis.plot_xdepth_list(nbins = 100, pids = [-14, 14, 2212, -13, 13], 
                                  xrange = (0, 1012), per_run = True)

In [ ]:
cascade_analysis.plot_xdepth_stop(nbins = 100, pids = [-14, 14, 2212, -13, 13, 22], #pids = None, 
                                  all_pids = True, 
                                  xrange = (0, 1168), per_run = False)

In [ ]:
cascade_analysis.plot_height_list(pids = [22, -13, 13, -11, 11], all_pids=True)

In [ ]:
cascade_analysis.digitize()

In [ ]:
import matplotlib.pylab as plt
plt.semilogx(cascade_analysis.egrid, cascade_analysis.hist_dict[22][0][99])



len(cascade_analysis.hist_dict[22][0])